In [16]:
import pyodbc
from datetime import date, datetime, timedelta
import pandas as pd
import win32com.client as win32
from databases import Bawm, Crm,PosicaoDm1Pickle, BDS, CVM
from funcoes_datas import FuncoesDatas
import os
import numpy as np
import xlwings as xw
import unicodedata
from itertools import zip_longest

In [17]:
fdt = FuncoesDatas()
crm = Crm()
produtos = crm.rating_produtos()
bds = BDS()
cvm = CVM()
dm1 = PosicaoDm1Pickle()

In [18]:
#Ajuste de produtos
produtos['GuidProduto']=produtos['GuidProduto'].str.lower()
produtos['CETIP']=produtos['CETIP'].str.lower()
produtos['ISIN']=produtos['ISIN'].str.lower()

#Produtos imobiliarios

pasta = 'O:/SAO/CICH_All/Real Estate/1. Users/Guilherme Liberatore/2. Real Estate/3. Monitoramento/PDFs'
ativos_imob = os.listdir(pasta)
ativos_imob = [ativo.split()[3].replace('.pdf','').lower() for ativo in ativos_imob]

pasta = 'O:/SAO/CICH_All/Renda Variável/Monitoramento/Monitoramentos/2024.1T'
ativos_rv = os.listdir(pasta)
ativos_rv  = [ativo.split()[3].replace('.pdf','').lower() for ativo in ativos_rv]

corporativos =  pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='Debentures')
corporativos  = corporativos[(corporativos['Status']=='Recomendado')|(corporativos['Status']=='Monitorado')]
corporativos['CETIP'] = corporativos['CETIP'].str.lower()
corporativos = list(corporativos['CETIP'])

estruturados = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='FIDC Recomendados')
estruturados = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='FIDC Recomendados')[['IdPAS']].astype('str')
estruturados = estruturados[estruturados['IdPAS'].notnull()].dropna()
estruturados = pd.merge(left=estruturados,right=produtos,on='IdPAS',how='left')

estruturados['CETIP'] = estruturados['CETIP'].str.lower()
estruturados['ISIN'] = estruturados['ISIN'].str.lower()
estruturado_cetip = list(estruturados['CETIP'])
estruturado_isin= list(estruturados['ISIN'])

#Gestores aprovados
gestores = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='Gestores')[['Gestora']]
gestores =list(gestores['Gestora'].str.lower())

#Códigos dos ativos aprovados

aprovados = ativos_rv + ativos_imob+corporativos+estruturado_cetip+estruturado_isin
aprovados = [aprovado for aprovado in aprovados if str(aprovado) != 'nan']

In [19]:
#Lista de Ativos já deliberados

#Retorna os dados da planilha de forma bruta (trata os ISINS / CODs para localizar no CRM)
produtos_comite = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Consolidado - Ativos Analisados v3.xlsx')
produtos_comite['Cetip/Isin/CNPJ']=produtos_comite['Cetip/Isin/CNPJ'].str.lower()
produtos_comite = produtos_comite.dropna(subset=['Cetip/Isin/CNPJ'])
produtos_comite = produtos_comite[produtos_comite['On/Off']!='OFF']
produtos_comite = produtos_comite[produtos_comite['Deliberação'].str.contains('Zerar')]
produtos_comite = produtos_comite.dropna(subset=['Cetip/Isin/CNPJ'])

#Vincula os dados informados  com o codigo crm
cnpj = pd.merge(left =produtos_comite,right=produtos,left_on='Cetip/Isin/CNPJ',right_on='CNPJ',how='left')
isin =  pd.merge(left = produtos_comite,right=produtos,left_on='Cetip/Isin/CNPJ',right_on='ISIN',how='left')
cetip = pd.merge(left = produtos_comite,right=produtos,left_on='Cetip/Isin/CNPJ',right_on='CETIP',how='left')

#Unifica as bases
ativos_deliberados = pd.concat([cnpj,isin,cetip])

#Trazer todos os codigos dos produtos deliberados
isin = ativos_deliberados[['Data Comitê','ISIN','Deliberação']].rename(columns={'ISIN':'CETIP / ISIN / CNPJ'})
cetip = ativos_deliberados[['Data Comitê','CETIP','Deliberação']].rename(columns={'CETIP':'CETIP / ISIN / CNPJ'})
cnpj = ativos_deliberados[['Data Comitê','CNPJ','Deliberação']].rename(columns={'CNPJ':'CETIP / ISIN / CNPJ'})
ativos_deliberados = pd.concat([cnpj,isin,cetip]).dropna(subset=['CETIP / ISIN / CNPJ'])

In [20]:
def remove_acentos(texto):
     return unicodedata.normalize('NFKD', texto).encode('ascii', 'ignore').decode('ascii')

In [21]:
#Função que analisa o emissor
def analise_emissor(ativos):
    ativos_bancos = ['lca','cdb','lci','lig','lf','dpge','lfs']
    coe = ['coe']
    bancos_aprovados = ['btg','brad','sant','bradesco','rabobank','santander','itaú','banco do brasil','bb','cef','caixa economica','bnds','nu','mercedez','bmg','bnp','agricole','safra','bbm','daycoval','votorantim','cnh','itaú','alfa','renault','volkswagen','itau','bv']
    ativos = ativos.split()
    if any(ativo in ativos for ativo in ativos_bancos):
        if any(ativo in ativos for ativo in bancos_aprovados):
            banco = 'Recomendado'
        else:
            banco = 'Reprovado'
            
    elif any(ativo in ativos for ativo in coe):
        banco = 'Reprovado'        
    else:
        banco = np.nan
    return banco      

In [22]:
#Função que analisa se o gestor é aprovado/Monitorado
def analise_gestor(ativos,gestores):
    gestores_aprovados = gestores
    ativos = ativos.split()
    if any(ativo in ativos for ativo in gestores):
        gestor = 'Recomendado'
    else:
        gestor = np.nan
    return gestor    

In [23]:
#Função que abre e-mail
def salva_arquivo(data_base):
    outlook = win32.Dispatch('outlook.application')
    mapi = outlook.GetNamespace("MAPI")
    inbox = mapi.GetDefaultFolder(6).Folders['Implementacao']
    messages = inbox.Items
    print('Processando ', len(messages), 'emails')
    send_dt = str(date.today())
    achei_arquivo = False
    for message in messages:
        message_dt = message.senton.date()
        print('Data do email:', message_dt, type(message_dt))
        assunto = message.subject
        print(assunto)
        assunto = assunto.lower()
        assunto = remove_acentos(assunto)
        if 'analise de ativos' in assunto:
            print(data_base, message_dt)
            if data_base <= message_dt:
                print('entrei')
                attachments = message.Attachments
                for att in attachments:
                    att_name = str(att).lower()
                    print(assunto)
                    assunto = assunto.replace('|','-')
                    if '.xlsx' in att_name:
                        cliente = att_name.split()[-1].replace('.xlsx','')
                        if os.path.exists(f'O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Análises Iniciais - Ativos/Análise de Ativos - Ativos Iniciais - {cliente} - Automático.xlsx'):
                            achei_arquivo = 'Arquivo já existente'
                        else:
                            att.SaveASFile(f'C:/Temp/Comitê/{assunto}.xlsx')
                            achei_arquivo = True                        
            else:
                print('não entrei')
    return achei_arquivo

In [24]:
def verifica_fundo_aberto(cnpj):
    try:
        tipo_cond = cvm.fundo_cadastro(cnpj)[['FORMA_DE_CONDOMINIO']][0]
    except:
        tipo_cond = np.nan
    return  tipo_cond
    

In [25]:
# Regras de Operação de uma transferencia
#Se for fundo exclusivo precisa ser transfer-in
#Se for tombamento vem tudo porque nós trazemos todos os ativos que estão na conta, ativo emitidos por tesouraria precisa ver se tem CETIP (LIG), senão não conseguimos trazer
#Se for STVM não conseguimos trazer fundo.
#STVM só podemos trazer o que possui cadastro na clearing CBLC e CETIP

def obs_ops(transferencia,ativos,tipo):
    ativos_bancos = ['lca','cdb','lci']
    ativos_tesouraria = ['lig']
    ativos_sem_liquidez = ['coe','fip']
    ativos = ativos.lower()
    ativos = ativos.split()
    if (transferencia =='Tombamento') and (any(ativo in ativos for ativo in ativos_bancos)):
            acao = 'Verificar se o ativo está cetipado' 
    elif any(ativo in ativos for ativo in ativos_sem_liquidez):
            acao = 'Teremos que manter até o vencimento'
    elif (any(ativo in ativos for ativo in  ativos_tesouraria)):
            acao = 'Não conseguimos trazer'
    elif (transferencia =='STVM') and (tipo =='fundo'):
            acao = 'Não conseguimos trazer'
    elif (transferencia =='STVM') and (any(ativo in ativos for ativo in ativos_bancos)):
            acao = 'Verificar se o ativo está cetipado'          
            
    else:
        acao = np.nan
         
    return acao          
             

In [26]:
# Salva o arquivo do e-mail e monta o dataframe
data = date.today()
data = data + timedelta(-1)
salva_arquivo(data)
mes = '{:02d}'.format(data.month) 

Processando  52 emails
Data do email: 2024-03-25 <class 'datetime.date'>
Análise de ativos | AraguaiaM
2024-06-23 2024-03-25
não entrei
Data do email: 2024-03-25 <class 'datetime.date'>
RES: Análise de ativos | AraguaiaM
2024-06-23 2024-03-25
não entrei
Data do email: 2024-03-25 <class 'datetime.date'>
RE: Análise de ativos | AraguaiaM
2024-06-23 2024-03-25
não entrei
Data do email: 2024-03-25 <class 'datetime.date'>
ENC: Análise de ativos | Sírio
2024-06-23 2024-03-25
não entrei
Data do email: 2024-03-25 <class 'datetime.date'>
ENC: Análise de ativos | Sírio
2024-06-23 2024-03-25
não entrei
Data do email: 2024-03-25 <class 'datetime.date'>
Análise de ativos | Sírio
2024-06-23 2024-03-25
não entrei
Data do email: 2024-03-27 <class 'datetime.date'>
FW: BTG PSF - Aprovação de Crédito Privado: Documentação Pendente
Data do email: 2024-03-28 <class 'datetime.date'>
Análise de ativos | Sírio
2024-06-23 2024-03-28
não entrei
Data do email: 2024-04-01 <class 'datetime.date'>
Carteira Prévia -

In [27]:
#Ajustas as colunas do dataframe

def analise_cliente(df,cliente,arquivo):
    PL = df.iat[2,1]
    info = df.loc[0:4,:1]
    colunas = df.iloc[6].to_list()
    df = df[7:1500]
    df.columns = colunas
    df = df.dropna(subset=['Cotista'])
    df['Ativo']= df['Ativo'].str.lower()
    df['Tipo']= df['Tipo'].str.lower()

    #Informações do cliente


    #Merge com o cadastro de produto
    cnpj = pd.merge(left =df,right=produtos,left_on='CETIP / ISIN / CNPJ',right_on='CNPJ',how='left')
    isin =  pd.merge(left = df,right=produtos,left_on='CETIP / ISIN / CNPJ',right_on='ISIN',how='left')
    cetip = pd.merge(left = df,right=produtos,left_on='CETIP / ISIN / CNPJ',right_on='CETIP',how='left')

    # #Unifica as bases
    Ativos = pd.concat([cnpj,isin,cetip])
    Ativos = Ativos.dropna(subset=['CETIP / ISIN / CNPJ'])
    Ativos = Ativos.dropna(subset=['IdPAS'])
    Ativos = Ativos[['Ativo_x','ISIN','CETIP','CNPJ','new_gestoridname','vencimento_y','Rating']].rename(columns={'Ativo_x':'Ativo','vencimento_y':'vencimento'})
    df = pd.merge(left=df, right=Ativos,on='Ativo',how='left').rename(columns={'vencimento_y':'vencimento'})
    df.to_excel('verificar.xlsx')

    #Separa os produtos bancarios

    df['Rating_Banco'] = df['Ativo'].apply(lambda x :analise_emissor(x))
    df = df[['CETIP / ISIN / CNPJ','PF / FE','Cotista','ON / OFF','Transferência','Origem','Ativo','Tipo','Valor Financeiro','% Patrimônio','ISIN',
       'CETIP','CNPJ','new_gestoridname','vencimento','Rating','Rating_Banco']].rename(columns={'Ativo_x':'Ativo','Rating':'Rating_CRM','new_gestoridname':'Gestor'})

    #Verifica se o ativo é monitorado ou recomendado pela equipe de investimentos
    lista_ativos_analisar=list(df['CETIP / ISIN / CNPJ'].str.lower())
    rating_investimentos = ['Recomendado' if ativo in aprovados else np.nan for ativo in lista_ativos_analisar ]
    df['Rating_Credito/RealEstate'] = rating_investimentos

    #Incluindo a recomendação do gestor
    lista_cnpj=list(df['CETIP / ISIN / CNPJ'])
    tipo_condominio = [verifica_fundo_aberto(x) for x in lista_cnpj]
    df['Tipo_Condominio'] = tipo_condominio

    #Verificar se o fundo é aberto
    lista_ativos_analisar=list(df['Ativo'].str.lower())
    rating_gestor = [analise_gestor(x,gestores) for x in lista_ativos_analisar]
    df['Rating_FOF'] = rating_gestor


    #Tudo o que for Selic será cadastrado como "Recomendado"
    df['Rating_Credito/RealEstate'].loc[df['Tipo']=='selic'] = 'Recomendado'

    #Deliberações Anteriores para o mesmo produto

    df = pd.merge(left=df, right=ativos_deliberados, on='CETIP / ISIN / CNPJ',how='left')
    df = df.drop(columns=['ISIN','CETIP','CNPJ'])

    #Atualiza o PL pelo financeiro
    df['% Patrimônio'] =df['Valor Financeiro'] / PL
    df['% Patrimônio'] = df['% Patrimônio']*100    
    df['Rating_FOF'].loc[df['Ativo'].str.contains('fip')] = 'Verificar FIP'
    df['Ativo'] = df['Ativo'].apply(lambda x : x.title())
    df['Risco'] = df.apply(lambda x: list([x['Rating_CRM'],x['Rating_Banco'],x['Rating_Credito/RealEstate'],x['Rating_FOF']]),axis=1)
    comite_list = list(df['Risco'])
    comite = []
    for x in comite_list:
        if ('Monitorado' in x) or ('Recomendado' in x):
            decisao = 'Risco Aprovado'
        else:
            decisao = 'Risco não aprovado sem liquidez'
        comite.append(decisao)    
    df['Risco']=comite
    df = df.drop_duplicates()
    df['Risco'] = df['Risco'].astype('str')
    df['Deliberação'] = df['Deliberação'].astype('str')
    df['Risco'].loc[df['Deliberação'].str.contains('Zerar')] = 'Zerar na Origem'
    

    #Incluindo as informações operacionais

    ops_obs=[]

    transferencias=list(df['Transferência'])
    ativos = list(df['Ativo'])
    tipos_ativos = list(df['Tipo'])

    for trans, ativo, tipo in zip_longest(transferencias, ativos, tipos_ativos, fillvalue="UNKNOWN"):
        ops_obs.append(obs_ops(trans,ativo,tipo))

    df['Obs_Operacionais'] = ops_obs
    df = df.replace({'nan': ''}, regex=True)


    return df, info

In [28]:
#Analisando todas os arquivos salvos nas pastas
pasta = 'C:/Temp/Comitê'
arquivos = os.listdir(pasta)

In [29]:
arquivos

['analise de ativos - ativos iniciais Slice.xlsx',
 'analise de ativos - ativos iniciais- CeluloseJ.xlsx',
 'analise de ativos - ativos iniciais.xlsx']

In [30]:
for arquivo in arquivos:
    if (arquivo.endswith('.xlsx')) and (arquivo != 'analise de ativos - ativos iniciais.xlsx'):
        arquivo = remove_acentos(arquivo)
        df = pd.read_excel(f'C:/Temp/Comitê/{arquivo}',header=None)
        cliente = arquivo.split()[-1].replace('.xlsx','')
        cliente = cliente.lower()
        dados = analise_cliente(df,cliente,arquivo)
        df = dados[0]
        info = dados[1]
        #Abre a planilha e cola os valores
        wb = xw.Book(f'C:/Temp/Comitê/analise de ativos - ativos iniciais.xlsx')
        sheet1 = wb.sheets[0]
        sheet1["B7"].options(pd.DataFrame, header=1, index=True, expand='table').value = df
        sheet1["A1"].options(pd.DataFrame, header=None, index=None, expand='table').value = info
        cliente = cliente.title()
        wb.save(f'O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Análises Iniciais - Ativos/Análise de Ativos - Ativos Iniciais - {cliente} - Automático.xlsx')
        wb.close()

In [31]:
emissores_fidc = pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='FIDC Recomendados')
emissores_fidc = emissores_fidc[['Emissor']].drop_duplicates()
emissores_corporativos =  pd.read_excel('O:/SAO/CICH_All/Investment Solutions/11. Comitê Exceção/Ativos Recomendados.xlsx',sheet_name='Debentures')
emissores_corporativos  = emissores_corporativos[(emissores_corporativos['Status']=='Recomendado')|(emissores_corporativos['Status']=='Monitorado')]
emissores_corporativos = emissores_corporativos[['Risco']].drop_duplicates().rename(columns={'Risco':'Emissor'})
emissores_aprovados = pd.concat([emissores_fidc,emissores_corporativos])

In [32]:
emissores_aprovados.to_excel('emissores_aprovados.xlsx')

In [33]:
#teste

df = pd.read_excel('C:/Temp/Comitê/Analise de Ativos - Ativos Iniciais - FE ACN.xlsx',header=None)
PL = df.iat[2,1]
info = df.loc[0:4,:1]
colunas = df.iloc[6].to_list()
df = df[7:1500]
# df.columns = colunas
# df = df.dropna(subset=['Cotista'])
# df['Ativo']= df['Ativo'].str.lower()
# df['Tipo']= df['Tipo'].str.lower()

# #Informações do cliente


# #Merge com o cadastro de produto
# cnpj = pd.merge(left =df,right=produtos,left_on='CETIP / ISIN / CNPJ',right_on='CNPJ',how='left')
# isin =  pd.merge(left = df,right=produtos,left_on='CETIP / ISIN / CNPJ',right_on='ISIN',how='left')
# cetip = pd.merge(left = df,right=produtos,left_on='CETIP / ISIN / CNPJ',right_on='CETIP',how='left')

# # #Unifica as bases
# Ativos = pd.concat([cnpj,isin,cetip])
# Ativos = Ativos.dropna(subset=['CETIP / ISIN / CNPJ'])
# Ativos = Ativos.dropna(subset=['IdPAS'])
# Ativos = Ativos[['Ativo_x','ISIN','CETIP','CNPJ','new_gestoridname','vencimento_y','Rating']].rename(columns={'Ativo_x':'Ativo','vencimento_y':'vencimento'})
# df = pd.merge(left=df, right=Ativos,on='Ativo',how='left').rename(columns={'vencimento_y':'vencimento'})
# df.to_excel('verificar.xlsx')

# #Separa os produtos bancarios

# df['Rating_Banco'] = df['Ativo'].apply(lambda x :analise_emissor(x))
# df = df[['CETIP / ISIN / CNPJ','PF / FE','Cotista','ON / OFF','Transferência','Origem','Ativo','Tipo','Valor Financeiro','% Patrimônio','ISIN',
#    'CETIP','CNPJ','new_gestoridname','vencimento','Rating','Rating_Banco']].rename(columns={'Ativo_x':'Ativo','Rating':'Rating_CRM','new_gestoridname':'Gestor'})

# #Verifica se o ativo é monitorado ou recomendado pela equipe de investimentos
# lista_ativos_analisar=list(df['CETIP / ISIN / CNPJ'].str.lower())
# rating_investimentos = ['Recomendado' if ativo in aprovados else np.nan for ativo in lista_ativos_analisar ]
# df['Rating_Credito/RealEstate'] = rating_investimentos

# #Incluindo a recomendação do gestor
# lista_cnpj=list(df['CETIP / ISIN / CNPJ'])
# tipo_condominio = [verifica_fundo_aberto(x) for x in lista_cnpj]
# df['Tipo_Condominio'] = tipo_condominio

# #Verificar se o fundo é aberto
# lista_ativos_analisar=list(df['Ativo'].str.lower())
# rating_gestor = [analise_gestor(x,gestores) for x in lista_ativos_analisar]
# df['Rating_FOF'] = rating_gestor


# #Tudo o que for Selic será cadastrado como "Recomendado"
# df['Rating_Credito/RealEstate'].loc[df['Tipo']=='selic'] = 'Recomendado'

# #Deliberações Anteriores para o mesmo produto

# df = pd.merge(left=df, right=ativos_deliberados, on='CETIP / ISIN / CNPJ',how='left')
# df = df.drop(columns=['ISIN','CETIP','CNPJ'])

# #Atualiza o PL pelo financeiro
# df['% Patrimônio'] =df['Valor Financeiro'] / PL
# df['% Patrimônio'] = df['% Patrimônio']*100    
# df['Rating_FOF'].loc[df['Ativo'].str.contains('fip')] = 'Verificar FIP'
# df['Ativo'] = df['Ativo'].apply(lambda x : x.title())
# # df['Risco'] = df.apply(lambda x: list([x['Rating_CRM'],x['Rating_Banco'],x['Rating_Credito/RealEstate'],x['Rating_FOF']]),axis=1)
# # comite_list = list(df['Risco'])
# # comite = []
# # for x in comite_list:
# #     if ('Monitorado' in x) or ('Recomendado' in x):
# #         decisao = 'Risco Aprovado'
# #     else:
# #         decisao = 'Risco não aprovado sem liquidez'
# #     comite.append(decisao)    
# # df['Risco']=comite
# # df = df.drop_duplicates()
# # df['Comites_Anteriores'] = df['Risco'].astype('str')
# # df['Deliberação'] = df['Deliberação'].astype('str')
# # df['Risco'].loc[df['Deliberação'].str.contains('Zerar')] = 'Zerar na Origem'

# # #Incluindo as informações operacionais

# # ops_obs=[]
    
# # transferencias=list(df['Transferência'])
# # ativos = list(df['Ativo'])
# # tipos_ativos = list(df['Tipo'])
 
# # for trans, ativo, tipo in zip_longest(transferencias, ativos, tipos_ativos, fillvalue="UNKNOWN"):
# #     ops_obs.append(obs_ops(trans,ativo,tipo))

# # df['Obs_Operacionais'] = ops_obs
# # df = df.replace({'nan': ''}, regex=True)


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Temp/Comitê/Analise de Ativos - Ativos Iniciais - FE ACN.xlsx'

In [ ]:
df

In [ ]:
fundos = fundos[~fundos['name'].str.contains('Alterado TI')]
fundos = fundos[~fundos['name'].str.contains('(TI Alterou)')]
fundos = fundos[~fundos['name'].str.contains('NÃO USAR')]

In [ ]:
dm1.lista_fundos_com_idcarteira(gestor_jbfo=True)

In [ ]:
()